In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [2]:
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                      exception_only=False, running_compiled_code=False):
       etype, value, tb = sys.exc_info()
       return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

In [ ]:
fname = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#10_final'}.xlsx")
cname = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#10_case'}.xlsx")
lname = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#10_law'}.xlsx")

dff = pd.read_excel(fname, index_col=0)
dfc = pd.read_excel(cname, index_col=0)
dfl = pd.read_excel(lname, index_col=0)

df = pd.concat([dff[['text', 'y_appeal']], dfc[1:], dfl[1:]], axis=1).reindex(dff.index)
df = df.dropna(subset=['y_appeal'])

In [ ]:
df_0 = df[df['y_appeal'] == 0].sample(frac=1)
df_1 = df[df['y_appeal'] == 1].sample(frac=1)

sample_size = len(df_0) if len(df_0) < len(df_1) else len(df_1)

df = pd.concat([df_0.head(sample_size), df_1.head(sample_size)]).sample(frac=1)

In [ ]:
X = df.drop('y_appeal', axis=1)
y = tf.keras.utils.to_categorical(df['y_appeal'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
X1_train = X_train[dfc.columns[1:]]
X1_test = X_test[dfc.columns[1:]]

X2_train = X_train[dfl.columns[1:]]
X2_test = X_test[dfl.columns[1:]]

In [ ]:
X3_train = list(X_train["text"])
X3_test = list(X_test["text"])

max_features = 30000
sequence_length = 256

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_features, split=' ', oov_token='<unw>')
tokenizer.fit_on_texts(X3_train)

X3_train = tokenizer.texts_to_sequences(X3_train)
X3_test = tokenizer.texts_to_sequences(X3_test)

X3_train = tf.keras.preprocessing.sequence.pad_sequences(X3_train, sequence_length)
X3_test = tf.keras.preprocessing.sequence.pad_sequences(X3_test, sequence_length)

In [ ]:
input_1 = tf.keras.Input(dtype = tf.float32, shape = (len(dfc.columns[1:]),))
input_2 = tf.keras.Input(dtype = tf.float32, shape = (len(dfl.columns[1:]),))

dense_layer_1_1 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(input_1)
dense_layer_1_2 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_1)
dense_layer_1_3 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_2)
dense_layer_1_4 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_3)
dropout_1_5 = tf.keras.layers.Dropout(rate = 0.5)(dense_layer_1_4)


dense_layer_2_1 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(input_2)
dense_layer_2_2 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_2_1)
dense_layer_2_3 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_2_2)
dense_layer_2_4 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_2_3)
dropout_2_5 = tf.keras.layers.Dropout(rate = 0.5)(dense_layer_2_4)

In [ ]:
embeddings_index = {}
f = open(os.path.join(os.path.dirname(os.getcwd()), 'data_processing', 'ft_0825_5.txt'),  encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

word_index = tokenizer.word_index

num_words = min(max_features, len(word_index)) + 1
print(num_words)
embedding_dim = 200
num_filters = 100

embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i > max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        embedding_matrix[i] = np.random.randn(embedding_dim)

In [ ]:
input_3 = tf.keras.Input(dtype = tf.float32, shape = (sequence_length,))
embedding_layer_3 = tf.keras.layers.Embedding(num_words,
                            embedding_dim,
                            embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                            input_length=sequence_length,
                            trainable=True)(input_3)

reshape_3 = tf.keras.layers.Reshape((sequence_length, embedding_dim, 1))(embedding_layer_3)

conv_0_3 = tf.keras.layers.Conv2D(num_filters, kernel_size=(3, embedding_dim), activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(3))(reshape_3)
conv_1_3 = tf.keras.layers.Conv2D(num_filters, kernel_size=(4, embedding_dim), activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(3))(reshape_3)
conv_2_3 = tf.keras.layers.Conv2D(num_filters, kernel_size=(5, embedding_dim), activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(3))(reshape_3)

maxpool_0_3 = tf.keras.layers.MaxPool2D(pool_size=(sequence_length - 3 + 1, 1), strides=(1,1), padding='valid')(conv_0_3)
maxpool_1_3 = tf.keras.layers.MaxPool2D(pool_size=(sequence_length - 4 + 1, 1), strides=(1,1), padding='valid')(conv_1_3)
maxpool_2_3 = tf.keras.layers.MaxPool2D(pool_size=(sequence_length - 5 + 1, 1), strides=(1,1), padding='valid')(conv_2_3)

concatenated_tensor_3 = tf.keras.layers.Concatenate(axis=1)([maxpool_0_3, maxpool_1_3, maxpool_2_3])
flatten_3 = tf.keras.layers.Flatten()(concatenated_tensor_3)
dropout_3 = tf.keras.layers.Dropout(rate = 0.5)(flatten_3)

In [ ]:
concat_layer = tf.keras.layers.Concatenate()([dropout_1_5, dropout_2_5, dropout_3])

dense_layer_3 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(concat_layer)
dense_layer_4 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_3)
dense_layer_5 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_4)
dense_layer_6 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_5)


output = tf.keras.layers.Dense(units = 2, activation = tf.nn.softmax)(dense_layer_4)

model = tf.keras.Model(inputs=[input_1, input_2, input_3], outputs=output)

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.SGD(learning_rate=0.0001), metrics=['acc'])
print(model.summary())

In [ ]:
history = model.fit(x=[X1_train, X2_train, X3_train], y=y_train, batch_size=2, epochs=100, verbose=1, validation_split=0.1)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

In [ ]:
score = model.evaluate(x=[X1_test, X2_test, X3_test], y=y_test, verbose=1)

print("Test Score:", score[0])
print("Test ACC:", score[1])

In [ ]:
#tf-idf 0.2
# 50-10-10-10-10 배치 2, 에폭 100, 0.01: 93 91 92// 60 55 52 55 53 52
# 50-10-10-10-10 배치 2, 에폭 100, 0.001: 73 73 76 72 72
# 50-10-10-10-10 배치 2, 에폭 100, 0.005: 90 65 91 87 52// 59 55 91 51 87 //48 50 94 81 56 // 82 94 53 54 58 => 70
# 50-10-10-10-10 배치 2, 에폭 100, 0.002: 60 90 93 59 63// 89 72 57 59 89 // 87 67 59 70 85 // 57 60 68 90 67 => 72

# 50-10-10-10-10 배치 2, 에폭 100, 0.0005: 74 91 88 76 88 // 75 77 90 92 71 // 76 90 93 79 76// 73 72 76 76 77 => 80
# 50-10-10-10-10 배치 4, 에폭 100, 0.0005: 79 70 74 78 76 // 82 72 84 78 77 // 85 75 77 84 77
# 50-10-10-10-10 배치 8, 에폭 100, 0.0005: 78 79 72 78 75 // 75 73 81 81 65
# 50-10-10-10-10 배치 16, 에폭 100, 0.0005: 81 75 76 69 77 // 

# 100-100-100-100-100 배치 2, 에폭 100, 0.0005: 69 95 93 76 79 // 73 94 79 75 77 // 78 70 75 60 88 // 69 92 88 86 74 => 79
# 10-10-10-10-10 배치 2, 에폭 100, 0.0005: 75 82 75 82 75 // 73 74 65 75 85
# 100-10-10-10-10 배치 2, 에폭 100, 0.0005: 86 85 80 72 78 // 96 84 73 65 79 // => 80

# 50-10-10-10-10 배치 2, 에폭 200, 0.0005: 72
# 50-10-10-10-10 배치 2, 에폭 100, 0.0005, random: 89 89 88 89 87 //89 88 88 89 89 
# 50-10-10-10-10 배치 2, 에폭 150, 0.0005, random: 87 87 87
# 50-10-10-10-10 배치 2, 에폭 200, 0.0005, random: 86 85 

# 50-10-10-10-10 배치 2, 에폭 100, 0.0005, random, w2v: 88 88 88

# 50-10-10-10 배치 2, 에폭 100, 0.0005, random, 판사없이: 90 88 89 89 89 // 88 89 
# 100-100-100-100 배치 2, 에폭 100, 0.0005, random, 판사없이: 85 
# 100-10-10-10 배치 2, 에폭 100, 0.0005, random, 판사없이: 88
# 8-8-8-8 배치 2, 에폭 100, 0.0005, random, 판사없이: 88
# 10-10-10-10 배치 2, 에폭 100, 0.0005, random, 판사없이: 87



# 50-10-10-10-10 배치 2, 에폭 100, 0.0001: 80 72 81 72 77 // 77 75 80 85 68 // 75
# 50-10-10-10-10 배치 2, 에폭 100, 0.00005: 85 80 75 77 73// 73 

# 50-10-10-10-10 배치 2, 에폭 100, 0.02: 51 50 53 54 45 56 54

#tf-idf 0.1
#50-10-10-10 배치 2, 에폭 100, 0.0005, random, 판사없이: 82



# 50-10-10-10/10-10 배치 2, 에폭 100, 0.0005, random, 판사없이: 87 87
# 50-10-10-10/10-10-10 배치 2, 에폭 100, 0.0005, random, 판사없이: 90 90 89 90 89 90 89 
# 50-10-10-10/10-10-10-10 배치 2, 에폭 100, 0.0005, random, 판사없이: 88 89

#new learning_rate=0.0005 에폭100: 72 // 에폭150: 75
#lr 0.001 에폭100: 78 75 
#lr 0.0001 74 
#lr 0.005 54 
#lr 0.002 71 
"""
#lr 0.001 에폭100: 78 75 78
#lr 0.0005 배치2 에폭100: 77
#lr 0.0005 배치32 에폭100: 74
#lr 0.0005 배치16 에폭100: 74 
#lr 0.0005 배치8 에폭100: 68
#lr 0.0005 배치4 에폭100:74

#lr 0.0005 배치2 10-10-10-(10) 에폭100: 78 
#lr 0.0005 배치2 100-100-100-(100) 에폭100:73
#lr 0.0005 배치2 200-200-200-(200) 에폭100: 73 
#lr 0.0005 배치2 10-10-10-(10) cv drop 0.2 에폭100:73
#lr 0.0005 배치2 10-(10-10-10) 에폭100: 75 
#lr 0.0005 배치2 10-10-10-(10) 전체 drop 0.5 에폭100: 80 76 77
#lr 0.0005 배치2 10-10-10-(10) 전체 drop 0.5 , l2=0.01 에폭100: 74
#lr 0.0005 배치2 10-10-10-(10) 전체 drop 0.5 , l2=0.1 에폭100: 75
#lr 0.0005 배치2 10-10-10-(10) 전체 drop 0.5 , l2=1 에폭100: 79 <=====loss 더 적은 걸로
#lr 0.0005 배치2 10-10-10-(10) 전체 drop 0.5 , l2=2 에폭100: 76
#lr 0.0005 배치2 10-10-10-(10) 전체 drop 0.5 , l2=4 에폭100: 80 <=====loss 더 적은 걸로
#lr 0.0005 배치2 10-10-10-(10) 전체 drop 0.5 , l2=3 , sequence 128 에폭100: 80 ????????????????????????? 더 해보기
#lr 0.0005 배치2 10*8-10*4-10*3 전체 drop 0.5, l2=3 에폭100: 74
#lr 0.0005 배치2 128*8-10*4-10*3 전체 drop 0.5, l2=3 에폭100: 76
#lr 0.0005 배치2 10*4-128*8-10*3 전체 drop 0.5, l2=3 에폭100: 76
#lr 0.0005 배치2 10*4-10*4-128*4 전체 drop 0.5, l2=3 에폭100: 74 

#lr 0.0005 배치2 10-10-10-(10) drop225 case 0.8 에폭100: 75 
#lr 0.0005 배치2 10*1-10*4-10*3 전체 drop 0.5 case 0.8 에폭100: 79 ?????????/ 절박하면 층 줄이기
#lr 0.0005 배치2 10*4-10*4-10*3 전체 drop 0.5 case 0.5 binary 에폭100: 75 
#lr 0.0005 배치2 10*1-10*1-10*1 전체 drop 0.5 case 0.5 에폭100: 78 
#lr 0.0005 배치2 10*1-10*1-10*1 max 1 전체 drop 0.5 case 0.5 에폭100: 74 
#lr 0.0002 배치2 10*4-10*4-10*4 drop225 case 0.5 에폭100: 70
#lr 0.0004 배치2 10*4-10*4-10*4 drop225 case 0.5 에폭100: 75
#lr 0.0006 배치2 10*4-10*4-10*4 drop225 case 0.5 에폭100: 71
#lr 0.0005 배치2 10*2-10*2-10*2 drop225 case 0.5 에폭100: 77
#lr 0.0005 배치2 10*16-10*16-10*2 drop225 case 0.5 에폭100:75 
#lr 0.0005 배치2 512*16-512*16-10*2 drop225 case 0.5 에폭100:73 
#lr 0.0005 배치2 512*16-512*16-512*16 drop225 case 0.5 에폭100:79

#lr 0.0005 배치2 1024*16-1024*16-1024*16 drop225 case 0.5 에폭100: 74----------------------------------


#lr 0.0005 배치2 256*16-256*16-10*2 drop225 case 0.5 에폭100: 78  
#lr 0.0005 배치2 256*16-256*16-256*2 drop225 case 0.5 에폭100:79  
#lr 0.0005 배치2 256*16-256*16-256*16 drop225 case 0.5 에폭100: 78

#lr 0.0005 배치2 128*16-128*16-10*2 drop225 case 0.5 에폭100: 75 
#lr 0.0005 배치2 128*16-128*16-128*2 drop225 case 0.5 에폭100: 79 
#lr 0.0005 배치2 128*16-128*16-128*16 drop225 case 0.5 에폭100: 75

#lr 0.0005 배치2 64*16-64*16-64*2 drop225 case 0.5 에폭100: 81 78 76 //에폭150: 76
#lr 0.0005 배치2 64*16-64*16-64*16 drop225 case 0.5 에폭100: 74

#lr 0.0005 배치2 100*16-100*16-100*2 drop225 case 0.5 에폭100: 77 78
#lr 0.0005 배치2 100*16-100*16-100*16 drop225 case 0.5 에폭100: 75

#lr 0.0005 배치2 32*16-32*16-32*2 drop225 case 0.5 에폭100: 75
#lr 0.0005 배치2 32*16-32*16-32*16 drop225 case 0.5 에폭100: 76

#lr 0.0005 배치2 10*16-10*16-10*16 drop225 case 0.5 에폭100: 학습안됨

#lr 0.0005 배치2 10-10-10-(10) 전체 drop 0.5 , l2=4 , sequence 128 에폭100:
#lr 0.0005 배치2 10-10-10-(10) 전체 drop 0.5 , l2=1 , sequence 128 에폭100:













#adam lr 0.001 배치2 10-10-10-(10) 전체 drop 0.5 에폭100: 학습안됨
#adam lr 0.0001 배치2 10-10-10-(10) 전체 drop 0.5 에폭100: 똑같음
#Adadelta lr 0.0001 배치2 10-10-10-(10) 전체 drop 0.5 에폭100: 학습안됨
#Adadelta lr 0.01 배치2 10-10-10-(10) 전체 drop 0.5 에폭100: 75
#Adamax lr 0.001 배치2 10-10-10-(10) 전체 drop 0.5 에폭100: 78 // loss 굿
#Adamax lr 0.0005 배치2 10-10-10-(10) 전체 drop 0.5 에폭100: 79
#Adamax lr 0.005 배치2 10-10-10-(10) 전체 drop 0.5 에폭100: 


#Nadam lr 0.001 배치2 10-10-10-(10) drop225 에폭100: 
#RMSprop lr 0.001 배치2 10-10-10-(10) 전체 drop 0.5 에폭100: 


#lr 0.0005 배치2 10*1-10*4-10*3 전체 drop 0.5 에폭100:







#lr 0.0005 배치2 10-10-10-(10) 전체 drop 0.5 에폭1000: 79
#lr 0.0005 배치2 10-10-10-(10) drop 0.5 0.2 0.5 에폭100: 76

"""



"""
Adadelta 0.001 에폭100:학습안됨
Adagrad 0.001 에폭100: 72
adam 0.001 에폭100: 
Adamax 0.001 에폭100:
Nadam 0.001 에폭100:
RMSprop 0.001 에폭100:
"""